# Training Vt with Simple MLP Regression 

We found saturation points and use assumption that Vt = Vgs-Vds in saturation point.\
By using that fact, we made Vt datasets.\
Now we'll train model that...\
Vt = F(W,L,T,Vgs,Vds,Ids)

In [45]:
import torch
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

## 1) Make dataset for Vt model

In [46]:
class VtDataset(torch.utils.data.Dataset):
    """
    Vt Dataset
    """
    def __init__(self, X, y, scale_data=False):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]

## 2) Make model

In [47]:
class VtMLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(4, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

## 3) Training

In [48]:
vt_df = pd.read_csv('../data_vt/vt_dataset_rev.csv')
vt_df.head()

,Vds,Vgs,Ids,W,L,T
0,0.1,1.0,2.555900e-08,10,5,0
1,1.0,0.6,2.024900e-08,10,5,0
2,5.0,0.6,2.271600e-08,10,5,0
3,10.0,0.6,2.948200e-08,10,5,0
4,20.0,0.4,2.438200e-08,10,5,0


In [49]:
vt_df_X = vt_df[['W', 'L', 'T', 'Vds']]
vt_X = vt_df_X.to_numpy()
vt_X

array([[1.0e+01, 5.0e+00, 0.0e+00, 1.0e-01],
       [1.0e+01, 5.0e+00, 0.0e+00, 1.0e+00],
       [1.0e+01, 5.0e+00, 0.0e+00, 5.0e+00],
       ...,
       [1.0e+03, 1.2e+01, 8.0e+01, 1.0e+01],
       [1.0e+03, 1.2e+01, 8.0e+01, 2.0e+01],
       [1.0e+03, 1.2e+01, 8.0e+01, 3.0e+01]])

In [50]:
vt_df_y = vt_df[['Vgs']]
vt_y = vt_df_y.to_numpy()
vt_y

array([[ 1.0000e+00],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 4.0000e-01],
       [-3.7581e-14],
       [ 1.0000e+00],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 4.0000e-01],
       [-3.7581e-14],
       [ 1.0000e+00],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 4.0000e-01],
       [-3.7581e-14],
       [ 1.0000e+00],
       [ 8.0000e-01],
       [ 8.0000e-01],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 4.0000e-01],
       [ 1.0000e+00],
       [ 8.0000e-01],
       [ 8.0000e-01],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 4.0000e-01],
       [ 1.0000e+00],
       [ 8.0000e-01],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 4.0000e-01],
       [ 1.0000e+00],
       [ 8.0000e-01],
       [ 8.0000e-01],
       [ 8.0000e-01],
       [ 6.0000e-01],
       [ 6.0000e-01],
       [ 1.0000e+00],
       [ 8.0000e-01],
       [ 8.0000e-01],
       [ 8

In [51]:
dataset = VtDataset(vt_X, vt_y)
print(len(dataset))
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=1)

432


In [52]:
mlp = VtMLP()

In [53]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-1)

In [54]:
# Run the training loop
for epoch in range(0, 100): # 20 epochs at maximum
    
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    # Iterate over the DataLoader for training data
    for i, data in enumerate(train_loader, 0):
        # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform forward pass
        outputs = mlp(inputs)
        print(outputs)

        # Compute loss
        loss = loss_function(outputs, targets)
        
        # Perform backward pass
        loss.backward()
        
        # Perform optimization
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 1000 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / (i+1)*32))
            current_loss = 0.0

# Process is complete.
print('Training process has finished.')

Starting epoch 1
tensor([[ 2.4339e-01],
        [ 9.5714e-01],
        [ 4.0746e-01],
        [-1.0444e+00],
        [ 4.8784e+00],
        [ 2.0619e-01],
        [ 5.5900e+00],
        [-4.7618e-01],
        [-5.1161e-01],
        [ 9.5766e-01],
        [ 5.9790e-01],
        [-8.9868e-01],
        [ 1.6161e+00],
        [-4.1279e-01],
        [ 1.4829e-03],
        [ 5.3061e+00],
        [ 5.6568e+00],
        [-6.8680e-02],
        [ 5.4592e+00],
        [ 1.8848e-01],
        [ 1.9911e-01],
        [ 4.3212e+00],
        [-8.4697e-01],
        [ 1.6775e+00],
        [-3.9839e-01],
        [ 4.6695e+00],
        [-1.5106e+00],
        [ 1.1558e+00],
        [ 5.3667e+00],
        [ 1.3140e-01],
        [ 5.6410e+00],
        [ 2.3160e+00]], grad_fn=<AddmmBackward0>)
Loss after mini-batch     1: 206.363
tensor([[ 7.1795e+02],
        [ 6.6907e+02],
        [ 1.2103e+01],
        [-4.1822e+01],
        [-1.4273e+01],
        [-1.3553e+01],
        [ 6.9312e+00],
        [ 6.7052e+02],

## 4) Saving...

In [55]:
torch.save(mlp.state_dict(), '../checkpoint/vt_mlp_model.pt')